# VISUALIZE OUTPUTS FROM ANALYSIS FUNCTIONS

In [ ]:
# init
import pandas as pd
import numpy as np
import importlib
import datetime
import matplotlib.pyplot as plt
import tTsTGrpUtils as tsutil
import utils_plots as uplots

In [ ]:
# Load data
importlib.reload(tsutil)
root = "/host/verges/tank/data/daniel/3T7T/z/outputs/"

pth_04a_raw = "04a_dl_maps_08Oct2025-104443.pkl"
pth_04b_parc = "04b_dl_maps_parcel_08Oct2025-110220.pkl"

pth_05a_winComp = "05a_winStudy_08Oct2025-110338.pkl" # z or w scores
pth_05b_winComp_ic = "05b_stats_winStudy_grp_08Oct2025-110654.pkl" # flipped

pth_05c_winD = "05c_stats_winD_08Oct2025-140529.pkl" # Has all data from above maps as well. D score btw px and ctrl z-scores
pth_05d_btwD = "05d_btwD_08Oct2025-153038.pkl" # difference in D scores between studies

"""
dl_04a_maps = tsutil.loadPickle(root + pth_04a_raw, verbose = True)
dl_04b_parc = tsutil.loadPickle(root + pth_04b_parc, verbose = True)

dl_05a_winComp = tsutil.loadPickle(root + pth_05a_winComp, verbose = True)
dl_05b_winComp_ic = tsutil.loadPickle(root + pth_05b_winComp_ic, verbose = True)

dl_05c_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)
dl_05d_btwD = tsutil.loadPickle(root + pth_05d_btwD, verbose = True)
"""

In [ ]:
index = 20
# print keys

dl_04a_itm = dl_04a_maps[index]
dl_04b_itm = dl_04b_parc[index]
dl_05a_itm = dl_05a_winComp[index]
dl_05b_itm = dl_05b_winComp_ic[index]
dl_05c_itm = dl_05c_winD[index]
#dl_05d_itm = dl_05d_btwD[index]

print("04a raw maps:\t\t", dl_04a_itm.keys())
print("04b parc maps:\t\t", dl_04b_itm.keys())
print("05a winStudy Z:\t\t", dl_05a_itm.keys())
print("05b winStudy ic:\t", dl_05b_itm.keys())
print("05c winStudy D:\t\t", dl_05c_itm.keys())
# print("05d D dif:\t\t", dl_05d_itm.keys())

In [ ]:
# raw to Z plotting
# TODO. Put below loops into function
importlib.reload(uplots)
import datetime
import os

reimport = True
if reimport:
    dl_05b_winComp_ic = tsutil.loadPickle(root + pth_05b_winComp_ic, verbose = True)
    foi = ['thickness', 'T1map', 'flair'] # features of interest
    dl_05b_winComp_ic = [itm for itm in dl_05b_winComp_ic if itm['feature'] in foi]
    print(len(dl_05b_winComp_ic))

# NOTE. Made to only accept z-scores. Would need to change savenaming methods to accept w-scores

dfs_raw = ['df_maps_parc_glsr_mean', 'df_maps_parc_dk25_mean']
save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/rawToZ/"
marks = True

dl_05b_iterate = dl_05b_winComp_ic

for index, item in enumerate(dl_05b_iterate): # iterate over all items

    item_txt = tsutil.printItemMetadata(item, return_txt = True)
    print(f"\n{item_txt}")

    item_sv_name = f"{item.get('study',None)}_{item.get('region', None)}_{item.get('feature', None)}_{item.get('label', None)}_{item.get('smth', None)}"
    
    ids_ctrl = item['ctrl_IDs']
    ids_tle_r = item['TLE_R_IDs']
    ids_tle_l = item['TLE_L_IDs']
    ids_tle = ids_tle_r + ids_tle_l
    print(f"\t{len(ids_ctrl)} CTRL | {len(ids_tle)} TLE [{len(ids_tle_r)} R, {len(ids_tle_l)} L]")

    for raw_key in dfs_raw:
        
        item_sv_name = f"{item_sv_name}_{raw_key}"

        now = datetime.datetime.now().strftime("%d%b%Y-%H%M%S")
        
        print(f"\n\t{raw_key}:")
        df_raw  = item.get(raw_key, None)
        if df_raw is None:
            print(f"\t\tKey not found. Skipping.")
            continue
        df_raw = tsutil.loadPickle(df_raw, verbose = False)

        df_raw_crtl = df_raw[df_raw.index.isin(ids_ctrl)]
        df_raw_ctrl_mn = df_raw_crtl.mean()
        df_raw_ctrl_std = df_raw_crtl.std()
        df_raw_ctrl_stats = pd.concat([df_raw_ctrl_mn, df_raw_ctrl_std], axis=1).T
        df_raw_ctrl_stats.index = ['mean', 'std']

        print(f"\t\tShape of all raw data: {df_raw.shape}")
        print(f"\t\tShape of raw_ctrls: {df_raw_crtl.shape}")

        # iterate over patients
        df_raw_grp = df_raw_crtl = df_raw[df_raw.index.isin(ids_tle)]
        print(f"\t\tShape of raw_grp: {df_raw_grp.shape}")
        z_key = f'{raw_key}_z'
        df_z_grp = item.get(z_key, None)
        if df_z_grp is None:
            print(f"\t\tKey not found: {z_key}. Skipping z-score part.")
            continue
        df_z = tsutil.loadPickle(df_z_grp, verbose = False)
        df_z_grp = df_z[df_z.index.isin(ids_tle)]
        print(f"\t\tShape of z_grp: {df_z_grp.shape}")

        # sort column names by L/R and increasing index number
        df_raw_ctrl_stats_srt = tsutil.sortCols(df_raw_ctrl_stats)
        df_raw_grp_srt = tsutil.sortCols(df_raw_grp)
        df_z_grp_srt = tsutil.sortCols(df_z_grp)

        # create the ctrl axis object since it is the same for all patients that follow. Pass this axis and add to it for each patient
        ctrl_fig_ax = uplots.get_ctrl_ax(df_raw_ctrl_stats_srt, ylbl = f"Raw {item.get('feature', None)} values", marks = marks)
        
        # show ctrl_fig
        figs = []
        for i, pid in enumerate(df_raw_grp_srt.index):
            save_name = f"{item_sv_name}_{index}_{i}_{now}"
            #print(f"\t\t\t{save_name}")
            
            fig_pth = uplots.rawToZ_plot_ctrl_ax(ctrl_fig_ax = ctrl_fig_ax,
                df_grp = df_raw_grp_srt.loc[pid].values,
                df_grp_z = df_z_grp_srt.loc[pid].values,
                id = pid,
                save_pth = f"{save_path}/raw",
                save_name = save_name,
                marks = marks,
                color_by_z = True)
            
            figs.append(fig_pth)

        # merge figures and deleted raw images
        uplots.pngs2pdf(fig_dir = f"{save_path}/raw", 
                        ptrn = item_sv_name,
                        output = save_path)
        # delete files in figs
        for f in figs:
            try:
                os.remove(f)
            except:
                pass
                

#df_raw_ctrl_stats

In [ ]:
# plot histograms with distribution of z-scores for patients and controls for each parcel, visualize d-scoring
# violin plot with distribution of PT and ctrl for each parcel.
# Plot D-score below these violin plots
importlib.reload(uplots)

dl_winD = tsutil.loadPickle(root + pth_05c_winD, verbose = True)
foi = ['thickness', 'T1map', 'flair'] # features of interest
dl_winD = [itm for itm in dl_winD if itm['feature'] in foi]
print(f"Len of dl: {len(dl_winD)}")

In [ ]:
# Plot z-scoring to D-scores
# TODO. Put below loops into function

importlib.reload(uplots)

dl_winD_iterate = dl_winD
save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/zToD" # will save raw images to ./raw
d_key = 'd_TLE_ic_ipsiTo-L' # index of cohen's d values in df_d
skip_idx = []

test = False
counter = 0
now = datetime.datetime.now().strftime("%d%b%Y-%H%M%S")
commonNames = set()

for i, item in enumerate(dl_winD_iterate):
    
    # find pair
    if i in skip_idx:
        continue

    idx_other = tsutil.get_pair(dl_winD, idx = i, mtch=['region', 'surf', 'label', 'feature', 'smth', 'parcellation'], skip_idx=[i])
    if idx_other is None:
        continue
    else:
        skip_idx += [i, idx_other]
        counter += 1
    
    if counter % 10 == 0:
        print(f"Progress: {counter} pairs of {len(dl_winD_iterate)//2}")

    tT_idx, sT_idx = tsutil.determineStudy(dl_winD, i, idx_other, study_key = 'study')
    
    tT_item = dl_winD[tT_idx]
    sT_item = dl_winD[sT_idx]

    txt_tT = tsutil.printItemMetadata(tT_item, return_txt = True)
    txt_sT = tsutil.printItemMetadata(sT_item, return_txt = True)
    title = tsutil.printItemMetadata(tT_item, return_txt = True, printStudy=False)
    
    commonName =  f"{tT_item.get('region', None)}_{tT_item.get('parcellation', None)}_{tT_item.get('feature', None)}_{tT_item.get('label', None)}"
    commonNames.add(commonName)
    
    save_name = f"{commonName}_{tT_item.get('smth', None)}_{now}"
    print(f"\n[{i}, {idx_other}]\t{title}")
    region = item.get('region', None)
    if region is not None:
        if region == 'hippocampus':
            key_str = 'dk25'
        elif region == 'cortex':
            key_str = 'glsr'
        else:
            ValueError(f"Region not recognized: {region}")
    else:
        ValueError("Region key not found.")

    # load dataframes
    dfs_d = []
    dfs_grp = []
    dfs_ctrl = []
    
    for itm_study in  [tT_item, sT_item]:

        df_d_ic = itm_study.get('df_d_ic', None)
        if df_d_ic is not None and isinstance(df_d_ic, str):
            df_d_ic = tsutil.loadPickle(df_d_ic, verbose = False)
        elif df_d_ic is None:
            print("WARNING. Key not found: df_maps_parc_d_score")
            continue
        else:
            assert isinstance(df_d_ic, pd.DataFrame), f"df_d not a dataframe nor string: {type(df_d_ic)}"
        #print(f"\tRetrieving d values for index {d_key}. [Available indices: {df_d_ic.index.tolist()}]")
        
        try:
            df_d_ic_srt = tsutil.sortCols(df_d_ic)
            d_ic = df_d_ic_srt.loc[d_key] # extract appropriate row from df_d
        except Exception as e:
            print(f"Type df_d_ic: {type(df_d_ic)}")
            print(f"WARNING. Could not retrieve d values for index {d_key}. Error: {e}")
            print(f"Indices:\n{df_d_ic.index.tolist()}")
            continue

        df_grp_ic_key = f"df_maps_parc_{key_str}_mean_z_TLE_ic"
        df_grp_ic = itm_study.get(df_grp_ic_key, None)
        if df_grp_ic is not None and isinstance(df_grp_ic, str):
            df_grp_ic = tsutil.loadPickle(df_grp_ic, verbose = False)
        elif df_grp_ic is None:
            print(f"WARNING. Key not found: {df_grp_ic_key}")
            continue
        else:
            assert isinstance(df_grp_ic, pd.DataFrame), f"df_grp not a dataframe nor string: {type(df_grp_ic)}"
        df_grp_ic = tsutil.sortCols(df_grp_ic)


        df_ctrl_key = f"df_maps_parc_{key_str}_mean_z_ctrl"
        df_ctrl = itm_study.get(df_ctrl_key, None)
        if df_ctrl is not None and isinstance(df_ctrl, str):
            df_ctrl = tsutil.loadPickle(df_ctrl, verbose = False)
        elif df_ctrl is None:
            ValueError(f"Key not found: {df_ctrl_key}")
        else:
            assert isinstance(df_ctrl, pd.DataFrame), f"df_ctrl not a dataframe nor string: {type(df_ctrl)}"

        # rename ctrl columns ipsi/contra
        ispiTo = itm_study.get('ipsiTo', None)
        df_ctrl_ic = df_ctrl.copy()
        if ispiTo is not None:
            if ispiTo == 'L':
                df_ctrl_ic.columns = [col.replace('_L', '_ipsi').replace('_R', '_contra') for col in df_ctrl_ic.columns]
            elif ispiTo == 'R':
                df_ctrl_ic.columns = [col.replace('_R', '_ipsi').replace('_L', '_contra') for col in df_ctrl_ic.columns]
            else:
                ValueError(f"ipsiTo not recognized: {ispiTo}")
        else:
            df_ctrl_ic.columns = [col.replace('_L', '_ipsi').replace('_R', '_contra') for col in df_ctrl_ic.columns]
            print("Warning: ipsiTo key not found. Defaulting to `L`.")
        df_ctrl_ic = tsutil.sortCols(df_ctrl_ic)

        
        dfs_d.append(d_ic)
        dfs_grp.append(df_grp_ic)
        dfs_ctrl.append(df_ctrl_ic)
        #print(f"\t[{item['study']}] d: {df_d_ic.shape}\tgrp: {df_grp_ic.shape}\tctrl: {df_ctrl_ic.shape}")

    if len(dfs_d) != 2 or len(dfs_grp) != 2 or len(dfs_ctrl) != 2:
        print("Warning: Did not find pairs for both studies. Skipping.")
        continue
    
    path = uplots.plot_zToD(df_grp = dfs_grp, df_ctrl = dfs_ctrl, df_d = dfs_d,
                     d_mode = 'rect', xlbl="Vertex/Parcel", ylbl="Z-Score", dlab="Cohen's D", title=title,
                     save_path = f"{save_path}/raw", save_name = save_name, verbose = True)
    
    if test and counter == 1:
        break

for commonName in commonNames:
    print(commonName)
    uplots.pngs2pdf(fig_dir = save_path, 
                    ptrn = commonName,
                    output = save_path)


In [ ]:
# Plot D-score differences between studies
importlib.reload(uplots)
importlib.reload(tsutil)

test = False
reimport = True
save_path = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/dD" # will save raw images to ./raw

if reimport:    
    dl_btwD = tsutil.loadPickle(root + pth_05d_btwD, verbose = True)

foi = ['thickness', 'T1map', 'flair'] # features of interest
dl_btwD = [itm for itm in dl_btwD if itm['feature'] in foi]
print(f"{len(dl_btwD)} items in dl")

uplots.btwD_vis(dl = dl_btwD, save_path = save_path,
                key_winD = 'df_d_ic', idx_winD = ['d_TLE_ic_ipsiTo-L'],
                key_btwD = 'comps_df_d_ic', idx_btwD = ['d_TLE_ic_ipsiTo-L_Δd', 'd_TLE_ic_ipsiTo-L_Δd_by3T', 'd_TLE_ic_ipsiTo-L_Δd_by7T'],
                test = test)
